In [1]:
import os
from pydantic import Extra
import requests
from typing import Any, List, Mapping, Optional
import datetime
from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.prompts import PromptTemplate
from langchain.agents import tool
from langchain.agents import create_react_agent, AgentExecutor, create_self_ask_with_search_agent
# Run chain
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import FAISS # type: ignore
from langchain_ollama import OllamaEmbeddings # type: ignore
import warnings
warnings.filterwarnings('ignore')

In [2]:
os.environ["token"] = "Bearer token|4353b65f-2b1b-4e15-9d6c-47bd024700ac|389496b55d294f224d491863705ea80c3b1e723be3183cf9ff3db246fee1ca03"
token = os.environ["token"]

In [3]:
from typing import ClassVar

parser = StrOutputParser()

class LlamaLLM(LLM):
    llm_url: ClassVar[str] = 'https://api.lab45.ai/v1.1/skills/completion/query'
    

    @property
    def _llm_type(self) -> str:
        return "gpt-35-turbo-16k"
    
    def _call(
        self,
        prompt: str,
        user: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")

        payload = {
        "messages": [
            {
            "content": prompt,
            "role": user
            }
        ],
        "skill_parameters": {
            "model_name": "gpt-35-turbo-16k",
            "max_output_tokens": 4096,
            "temperature": 0,
            "top_k": 5
        },
        "stream_response": False
        }

        headers = {"Content-Type": "application/json","Authorization": token}

        response = requests.post(self.llm_url, json=payload, headers=headers, verify=False)

       # print("API Response:", response.json())
        response.raise_for_status()

        return response.json()  # get the response from the API

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {
            "llmUrl": self.llm_url
            }
    
llm = LlamaLLM()

In [4]:
messages = [
    SystemMessage(content="Translate the following from English into Kannada"),
    HumanMessage(content="Hi"),
]

In [5]:
system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [6]:
formatted_prompt = prompt_template.format(language="kannada", text="How are you?")
result = llm._call(prompt=formatted_prompt, user="system")
parser.parse(result)

{'data': {'content': 'ನೀವು ಹೇಗಿದ್ದೀರಿ?'}}

In [7]:
@tool
def get_system_time(format: str = "%Y-%m-%d %H:%M:%S"):
    """ Returns the current date and time in the specified format """

    current_time = datetime.datetime.now()
    formatted_time = current_time.strftime(format)
    return formatted_time

tools =[get_system_time]

In [8]:
messages = [
    SystemMessage(content="You are Question and Answering System?"),
    HumanMessage(content="What is the current date and time?"),
]

In [9]:
system_template = "What is the current time? using the tools {tools} and 5 and half hours to it:"
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{tools}")]
)

In [10]:
formatted_prompt = prompt_template.format(tools = [get_system_time])
result = llm._call(prompt=formatted_prompt, user="system")
parser.parse(result)

{'data': {'content': 'The current time is 10:30 AM. Adding 5 and a half hours to it, the result would be 4:00 PM.'}}

In [ ]:
@tool
def get_system_time(format: str = "%Y-%m-%d %H:%M:%S"):
    """ Returns the current date and time in the specified format """

    current_time = datetime.datetime.now()
    formatted_time = current_time.strftime(format)
    return formatted_time

tools = [get_system_time]
query = "What is the current time in London? (You are in India). Just show the current time and not the date"

prompt_template = PromptTemplate(
    input_variables=["agent_scratchpad", "input", "tool_names", "tools"],
    template="""Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""
)

agent = create_react_agent(llm, tools, prompt_template)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


#agent_executor.invoke({"input": query})
result = agent_executor.invoke({"input": query, "user": "system"})
